# From ATOM feed to pandas

In [13]:
import pandas as pd
import feedparser

In [14]:
# Earthquakes for one month period to today
#
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.atom"
feed = feedparser.parse(url)

In [15]:
from typing import Any

entries: list[dict[str, Any]] = [dict(e) for e in feed.entries]
entries[0]

{'id': 'urn:earthquake-usgs-gov:ci:41131255',
 'guidislink': True,
 'link': 'https://earthquake.usgs.gov/earthquakes/eventpage/ci41131255',
 'title': 'M 1.0 - 8 km ENE of Big Bear City, CA',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.atom',
  'value': 'M 1.0 - 8 km ENE of Big Bear City, CA'},
 'updated': '2025-12-03T00:02:59.044Z',
 'updated_parsed': time.struct_time(tm_year=2025, tm_mon=12, tm_mday=3, tm_hour=0, tm_min=2, tm_sec=59, tm_wday=2, tm_yday=337, tm_isdst=0),
 'links': [{'rel': 'alternate',
   'type': 'text/html',
   'href': 'https://earthquake.usgs.gov/earthquakes/eventpage/ci41131255'}],
 'summary': '<dl><dt>Time</dt><dd>2025-12-03 00:00:57 UTC</dd><dd>2025-12-03 00:00:57 UTC at epicenter</dd><dt>Location</dt><dd>34.287&deg;N 116.768&deg;W</dd><dt>Depth</dt><dd>7.47 km (4.64 mi)</dd></dl>',
 'summary_detail': {'type': 'text/html',
  'language': None,
  'base': 'https://earthquak

In [16]:
df_quakes = pd.json_normalize(entries)
df_quakes.head(2)

,id,guidislink,link,title,updated,updated_parsed,links,summary,georss_elev,tags,title_detail.type,title_detail.language,title_detail.base,title_detail.value,summary_detail.type,summary_detail.language,summary_detail.base,summary_detail.value,where.type,where.coordinates
0,urn:earthquake-usgs-gov:ci:41131255,True,https://earthquake.usgs.gov/earthquakes/eventp...,"M 1.0 - 8 km ENE of Big Bear City, CA",2025-12-03T00:02:59.044Z,"(2025, 12, 3, 0, 2, 59, 2, 337, 0)","[{'rel': 'alternate', 'type': 'text/html', 'hr...",<dl><dt>Time</dt><dd>2025-12-03 00:00:57 UTC</...,-7469.9997901917,"[{'term': 'Past Hour', 'scheme': None, 'label'...",text/plain,None,https://earthquake.usgs.gov/earthquakes/feed/v...,"M 1.0 - 8 km ENE of Big Bear City, CA",text/html,None,https://earthquake.usgs.gov/earthquakes/feed/v...,<dl><dt>Time</dt><dd>2025-12-03 00:00:57 UTC</...,Point,"(-116.76766967773, 34.287334442139)"
1,urn:earthquake-usgs-gov:nc:75273356,True,https://earthquake.usgs.gov/earthquakes/eventp...,"M 1.1 - 8 km NW of The Geysers, CA",2025-12-02T23:57:13.573Z,"(2025, 12, 2, 23, 57, 13, 1, 336, 0)","[{'rel': 'alternate', 'type': 'text/html', 'hr...",<dl><dt>Time</dt><dd>2025-12-02 23:55:34 UTC</...,-2039.999961853,"[{'term': 'Past Hour', 'scheme': None, 'label'...",text/plain,None,https://earthquake.usgs.gov/earthquakes/feed/v...,"M 1.1 - 8 km NW of The Geysers, CA",text/html,None,https://earthquake.usgs.gov/earthquakes/feed/v...,<dl><dt>Time</dt><dd>2025-12-02 23:55:34 UTC</...,Point,"(-122.80999755859, 38.830501556396)"


1. How many quakes were there?

In [17]:
# Quake count
len(df_quakes)


8599

In [18]:
# Add a mag column containing the magnitude of the quake extracted from title using regex
#
mag = (
    df_quakes["title"]
    .str.extract(r"\bM\s*([0-9]+(?:\.[0-9]+)?)")[0]
    .astype(float)
)

# mag column now holds, for example, 2.3 for "M 2.3 - 11 km SSE of Maria Antonia, Puerto Rico."
#
df_quakes["mag"] = mag
df_quakes.head(1)

,id,guidislink,link,title,updated,updated_parsed,links,summary,georss_elev,tags,...,title_detail.language,title_detail.base,title_detail.value,summary_detail.type,summary_detail.language,summary_detail.base,summary_detail.value,where.type,where.coordinates,mag
0,urn:earthquake-usgs-gov:ci:41131255,True,https://earthquake.usgs.gov/earthquakes/eventp...,"M 1.0 - 8 km ENE of Big Bear City, CA",2025-12-03T00:02:59.044Z,"(2025, 12, 3, 0, 2, 59, 2, 337, 0)","[{'rel': 'alternate', 'type': 'text/html', 'hr...",<dl><dt>Time</dt><dd>2025-12-03 00:00:57 UTC</...,-7469.9997901917,"[{'term': 'Past Hour', 'scheme': None, 'label'...",...,None,https://earthquake.usgs.gov/earthquakes/feed/v...,"M 1.0 - 8 km ENE of Big Bear City, CA",text/html,None,https://earthquake.usgs.gov/earthquakes/feed/v...,<dl><dt>Time</dt><dd>2025-12-03 00:00:57 UTC</...,Point,"(-116.76766967773, 34.287334442139)",1.0


2. Filter df_quakes for mag 4.0 or larger and write the Title, Magnitude, (Longitude, Latitude) to a csv named data\quakes_ge_4.csv

In [21]:
quakes_filtered = df_quakes[df_quakes["mag"] >= 4.0]

df_csv = quakes_filtered[['title', 'mag', 'where.coordinates']]

df_csv.to_csv('data\quakes_ge_4.csv', index=False)